In [1]:
import tensorflow as tf
from datasets import load_dataset

# 定义数据集

In [51]:
dataset = load_dataset(path="seamew/ChnSentiCorp")

Using custom data configuration default
Reusing dataset chn_senti_corp (/Users/luominzhi/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/3 [00:00<?, ?it/s]

# 加载字典和分词工具

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [14]:
tokenized_ds = tokenizer.batch_encode_plus(
    batch_text_or_text_pairs=dataset["text"],
    truncation=True,
    padding="max_length",
    max_length=250,
    add_special_tokens=True,
    return_tensors="tf",
    return_length=True,
    # return_token_type_ids=True,
)

In [17]:
tokenizer.decode(tokenized_ds["input_ids"][0])

'[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 ， 有 电 动 扶 梯 直 接 到 达 海 边 ， 周 围 餐 馆 、 食 廊 、 商 场 、 超 市 、 摊 位 一 应 俱 全 。 酒 店 装 修 一 般 ， 但 还 算 整 洁 。 泳 池 在 大 堂 的 屋 顶 ， 因 此 很 小 ， 不 过 女 儿 倒 是 喜 欢 。 包 的 早 餐 是 西 式 的 ， 还 算 丰 富 。 服 务 吗 ， 一 般 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [22]:
tokenized_ds["input_ids"]

<tf.Tensor: shape=(20, 250), dtype=int32, numpy=
array([[ 101, 6848, 2885, ...,    0,    0,    0],
       [ 101, 8115,  119, ...,    0,    0,    0],
       [ 101, 2791, 7313, ...,    0,    0,    0],
       ...,
       [ 101, 2523, 1962, ...,    0,    0,    0],
       [ 101, 4023,  778, ...,    0,    0,    0],
       [ 101, 1079, 2100, ...,    0,    0,    0]], dtype=int32)>

In [21]:
tf.constant(dataset["label"])

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
      dtype=int32)>

# 构建训练 dataset

In [52]:
tokenized_ds = dataset.map(
    lambda example: tokenizer(example["text"], padding="max_length", truncation=True, max_length=200)
)

Loading cached processed dataset at /Users/luominzhi/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-bd19e88c8423571b.arrow


  0%|          | 0/1200 [00:00<?, ?ex/s]

  0%|          | 0/1200 [00:00<?, ?ex/s]

In [53]:
small_train_ds = tokenized_ds["train"].shuffle(seed=42).select(range(1000))
small_test_ds = tokenized_ds["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /Users/luominzhi/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-e1da0efa7158ae11.arrow


In [69]:
small_train_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [45]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [120]:
tf_train_ds = small_train_ds.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids",],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_ds = small_test_ds.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids",],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [82]:
next(iter(tf_train_ds))

({'input_ids': <tf.Tensor: shape=(8, 200), dtype=int64, numpy=
  array([[ 101, 1355, 4178, ...,    0,    0,    0],
         [ 101, 6843, 4638, ...,    0,    0,    0],
         [ 101, 6821, 3315, ...,    0,    0,    0],
         ...,
         [ 101,  702,  782, ...,    0,    0,    0],
         [ 101, 6841, 1217, ...,    0,    0,    0],
         [ 101, 1762, 6821, ...,    0,    0,    0]])>,
  'token_type_ids': <tf.Tensor: shape=(8, 200), dtype=int64, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])>,
  'attention_mask': <tf.Tensor: shape=(8, 200), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]])>},
 <tf.Tensor: shape=(8,), dtyp

# 加载预训练模型

In [31]:
from transformers import TFBertModel
pretrained = TFBertModel.from_pretrained("bert-base-chinese")

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [41]:
pretrained.summary()

Model: "tf_bert_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  102267648 
                                                                 
Total params: 102,267,648
Trainable params: 102,267,648
Non-trainable params: 0
_________________________________________________________________


# 模型试算

In [84]:
single_batch = next(iter(tf_train_ds))

out = pretrained(
    input_ids=single_batch[0]["input_ids"],
    attention_mask=single_batch[0]["attention_mask"],
    token_type_ids=single_batch[0]["token_type_ids"],
)

print(out)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(8, 200, 768), dtype=float32, numpy=
array([[[ 1.1308393 ,  0.2345363 , -0.8766137 , ..., -0.26060915,
         -0.04310881, -0.58737534],
        [-0.1569882 ,  0.56958866,  0.6289089 , ..., -0.768068  ,
         -0.11376459,  0.35377085],
        [ 0.78638947,  0.5999495 ,  0.64621556, ...,  0.2696762 ,
          1.5119927 , -0.22727573],
        ...,
        [-0.52723414, -0.24690893, -0.53745407, ...,  0.35378206,
          0.8578799 , -0.93970907],
        [-0.43473202,  0.46251753, -0.1544686 , ...,  0.4311419 ,
          0.5793288 , -1.2472293 ],
        [ 0.54778236,  0.44366223, -0.22687349, ...,  0.06052589,
          0.08866881, -0.4830872 ]],

       [[-0.20123546, -0.10884465,  0.2623894 , ..., -0.3264447 ,
          0.29584184, -0.13884443],
        [-0.01765029,  0.5939071 , -0.41618973, ..., -0.9018427 ,
         -0.16438168,  0.02866795],
        [ 1.4683472 ,  0.08501804, -1.3277123 , .

In [85]:
out.last_hidden_state.shape

TensorShape([8, 200, 768])

In [88]:
%timeit pretrained(input_ids=single_batch[0]["input_ids"], attention_mask=single_batch[0]["attention_mask"], token_type_ids=single_batch[0]["token_type_ids"],)

88.4 ms ± 290 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 定义下游任务模型

In [30]:
import tensorflow as tf

In [117]:
class Model(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__(*args, **kwargs)
        self.fc = tf.keras.layers.Dense(2)

    def call(self, inputs, training=None, mask=None):
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        x = pretrained(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        x = self.fc(x.pooler_output)

        return x


model = Model()

In [95]:
model.build()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [119]:
tf.argmax(model(next(iter(tf_train_ds))[0]), axis=-1)

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 1, 0, 1, 0, 1, 1, 0])>

In [136]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

model.fit(tf_train_ds, validation_data=tf_test_ds, epochs=3)

Epoch 1/3


2022-08-28 20:05:27.920914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - ETA: 0s - loss: 0.6219 - accuracy: 0.6620

2022-08-28 20:05:59.607706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 61s 426ms/step - loss: 0.6219 - accuracy: 0.6620 - val_loss: 0.4502 - val_accuracy: 0.8360
Epoch 2/3
125/125 [==============================] - 42s 334ms/step - loss: 0.4815 - accuracy: 0.7740 - val_loss: 0.4082 - val_accuracy: 0.8380
Epoch 3/3
125/125 [==============================] - 42s 336ms/step - loss: 0.4625 - accuracy: 0.7940 - val_loss: 0.3963 - val_accuracy: 0.8350
